# Naive Bayes

Notes: https://github.com/daviskregers/notes/blob/master/data-science/04-machine-learning-with-python/03-bayesian-methods.md

In [3]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('data/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('data/emails/ham', 'ham'))


In [4]:
data.head()

,message,class
data/emails/spam/00477.24ef7a042f97482f884387c75249380c,"<html>\n\n<TABLE id=AutoNumber2 style=""BORDER-...",spam
data/emails/spam/00414.b2312673ca5358901c801eb44c00e310,"<HTML>\n\n<PRE>\n\nDear Valued Member,\n\n\n\n...",spam
data/emails/spam/00303.22239f1393297a691eb5df3dfe7a5001,------=_NextPart_000_00E4_17A73C2D.E7104E07\n\...,spam
data/emails/spam/00103.2eef38789b4ecce796e7e8dbe718e3d2,Protect your financial well-being.\n\nPurchase...,spam
data/emails/spam/00079.cc3fa7d977a44a09d450dde5db161c37,Dear Sir / Madam\n\n\n\nIf you are fed up of b...,spam


Now we can use CountVecotorizer to split up each message into word lists and use the MultinomialNB classifier.

In [5]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB()

In [6]:
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], dtype='<U4')